In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
import scipy.signal as signal
import skimage
import skimage.exposure as expo
from skimage.io import imread
from skimage import data
from skimage import filters
from skimage import morphology
from skimage import color
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.color import label2rgb

lego=imread("https://github.com/srivol/visionindus/raw/master/legoFondnoir1.JPG")
plt.imshow(lego)
plt.title('Image originale')

In [ ]:
#legoFondnoir1.JPG : 70,50,50
#legoFondnoir2.JPG : 70,50,50
#legoFondnoir.jpg : 70,50,50
sR=160
sG=170
sB=140

legoR=lego[:,:,0]
legoG=lego[:,:,1]
legoB=lego[:,:,2]
hR,binsR=np.histogram(legoR,256,range=(0,255))
hG,binsG=np.histogram(legoG,256,range=(0,255))
hB,binsB=np.histogram(legoB,256,range=(0,255))
legoBINR=legoR>=sR
legoBING=legoG>=sG
legoBINB=legoB>=sB

plt.figure(figsize=(15, 15))
plt.subplot(331)
plt.imshow(legoR, cmap=plt.cm.gray)
plt.title('Canal rouge')
plt.subplot(332)
plt.imshow(legoG, cmap=plt.cm.gray)
plt.title('Canal vert')
plt.subplot(333)
plt.imshow(legoB, cmap=plt.cm.gray)
plt.title('Canal bleu')
plt.subplot(334)
plt.plot(binsR[:-1], hR,)
plt.title('Canal rouge')
plt.subplot(335)
plt.plot(binsG[:-1], hG)
plt.title('Canal vert')
plt.subplot(336)
plt.plot(binsB[:-1], hB)
plt.title('Canal bleu')
plt.subplot(337)
plt.imshow(legoBINR, cmap=plt.cm.gray)
plt.title('Canal rouge')
plt.subplot(338)
plt.imshow(legoBING, cmap=plt.cm.gray)
plt.title('Canal vert')
plt.subplot(339)
plt.imshow(legoBINB, cmap=plt.cm.gray)
plt.title('Canal bleu')
plt.show()

In [ ]:
plt.imshow(lego)
plt.title('Image originale')

legoblanc=np.zeros(legoBINR.shape)
legoblanc=legoBINR.astype(np.float)*legoBING*legoBINB
legoblanc=legoblanc.astype(np.uint8)

legojaune=np.zeros(legoBINR.shape)
legojaune=legoBINR.astype(np.float)*legoBING-legoblanc.astype(np.float)
legojaune=legojaune.astype(np.uint8)

legorouge=np.zeros(legoBINR.shape)
legorouge=legoBINR.astype(np.float)-legojaune.astype(np.float)-legoblanc.astype(np.float)
legorouge=legorouge.astype(np.uint8)

legobleu=np.zeros(legoBINR.shape)
legobleu=legoBINB-legoblanc
legobleu=legobleu.astype(np.uint8)

masque=np.ones((9,9))
legoblanc=ndi.morphology.binary_fill_holes(legoblanc,structure=masque)
legojaune=ndi.morphology.binary_fill_holes(legojaune,structure=masque)
legorouge=ndi.morphology.binary_fill_holes(legorouge,structure=masque)
legobleu=ndi.morphology.binary_fill_holes(legobleu,structure=masque)
legoblanc=skimage.morphology.remove_small_objects(legoblanc,min_size=2000)
legojaune=skimage.morphology.remove_small_objects(legojaune,min_size=2000)
legorouge=skimage.morphology.remove_small_objects(legorouge,min_size=2000)
legobleu=skimage.morphology.remove_small_objects(legobleu,min_size=2000)

plt.figure(figsize=(10, 10))
plt.subplot(221)
plt.imshow(legoblanc, cmap=plt.cm.gray)
plt.title('Lego blanc')
plt.subplot(222)
plt.imshow(legojaune, cmap=plt.cm.gray)
plt.title('Lego jaune')
plt.subplot(223)
plt.imshow(legorouge, cmap=plt.cm.gray)
plt.title('Lego rouge')
plt.subplot(224)
plt.imshow(legobleu, cmap=plt.cm.gray)
plt.title('Lego bleu')

plt.show()

In [ ]:
plt.imshow(lego)
plt.title('Image originale')
plt.show()

legoLABEL=np.zeros(lego.shape)

legoblanc=50*legoblanc
legojaune=100*legojaune
legorouge=150*legorouge
legobleu=200*legobleu

legoLABELcouleur=legoblanc+legojaune+legorouge+legobleu
legoLABELcouleur.astype(np.uint8)

plt.imshow(legoLABELcouleur, cmap=plt.cm.gray)
plt.title('Etiquetage 0 à 4 en fonction de la couleur de l objet')
plt.show()

rgb=color.label2rgb(legoLABELcouleur, lego, kind='avg')
plt.imshow(rgb)
plt.show()

imageio.imsave('legoTriCouleur.jpg',rgb)